MODEL USED

In [1]:
MODEL = "llama3.2"

In [2]:
import pydantic
print(pydantic.__version__)


2.10.6


Initialization of model and Embeddings

In [11]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
# model.invoke("where are you prividing your response from")


Document loaded and splitted

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./paper-crash-detection.pdf")
pages = loader.load_and_split()

In [51]:
# pages

creating prompt for LLM

In [22]:
from langchain.prompts import PromptTemplate

template = """
Answer the questions based on the context below. If you cannot answer the question given, just reply I don't know

Context: {context}
Question: {question}

"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="", question=""))


Answer the questions based on the context below. If you cannot answer the question given, just reply I don't know

Context: 
Question: 




In [23]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

creating a chain 

In [24]:
chain = prompt | model | parser

In [25]:
chain.invoke(
    {
        "context":"nepal", 
        "question":"what is the country name"
    }
)

'The country name in Nepal is "Nepal".'

storing embeddings in vectorestores

In [26]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embeddings
)

creating retreiver

In [28]:
retreiver = vectorstore.as_retriever()

In [29]:
retreiver.invoke("GPS module")

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-04-30T11:04:15+05:30', 'author': 'Microsoft account', 'moddate': '2024-04-30T11:04:15+05:30', 'source': './paper-crash-detection.pdf', 'total_pages': 13, 'page': 5, 'page_label': '6'}, page_content="Nabaraj Subedi, Nirajan Paudel, Manish Chhetri, Sudarshan Acharya, Nabin Lamichhane  \nJournal of IoT in Social, Mobile, Analytics, and Cloud, March 2024, Volume 6, Issue 1 59 \n \nFigure 3. Crash Detection Flow Chart \n3.3 Components Used \na.   Arduino UNO  \nThe Arduino Uno, built around the ATmega328P microcontroller, receives velocity and \ntilt values from the accelerometer [7]. It verifies if these values meet predefined conditions \nbefore proceeding with further operations. \nb.   MPU 6050 \nIn this study, a 10 DOF IMU Sensor, capable of measuring 3 -axis accelerometer data, \nis utilized to detect changes in a vehicle's orientation, indicative of potential accidents. To

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retreiver,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [33]:
response=chain.invoke({"question":"who are the authors of this academic paper?"})

In [34]:
print(response)

The authors of this academic paper are:

1. Nabaraj Subedi
2. Nirajan Paudel
3. Manish Chhetri
4. Sudarshan Acharya
5. Nabin Lamichhane


In [37]:
# to stream the answers 

for s in chain.stream({"question":"who are the authors of this academic paper?"}):
    print(s, end="", flush=True)

The authors of this academic paper are:

1. Nabaraj Subedi
2. Nirajan Paudel
3. Manish Chhetri
4. Sudarshan Acharya
5. Nabin Lamichhane